# Scenario 1: State tax board

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and wants a report of current liquor sales by county and projections for the rest of the year.

**Goal for Scenario #1:** Your task is as follows:

* Calculate the yearly liquor sales for each score using the provided data. You can add up the transactions for each year, and store sales in 2015 specifically will be used later as your target variable.
* Use the data from 2015 to make a linear model using as many variables as you find useful to predict the yearly sales of each store. You must use the sales from Jan to March per store as one of your variables.
* Use your model for 2015 to estimate total sales for each store in 2016, extrapolating from the sales so far for Jan-March of 2016.
* Report your findings, including any projected increase or decrease in total sales (over the entire state) for the tax committee of the Iowa legislature.
* Use cross-validation to check how your model predicts to held out data compared to the model metrics on the full dataset.
* _Challenging Bonus_: We did not cover the topics of regularization for linear regression this week, but those feeling bold can try to use and understand regularizing linear regressions. This will require self-guided research/reading and scikit-learn functions that we have not gone over in class! Use cross-validation to tune the regularization parameter that maximizes R^2 on your holdout sets for the Ridge regression and the Lasso Regression. Do the regularized models perform better than the non-regularized model? Which regularized model performs better? What is the Ridge regression doing? What is the Lasso doing.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline

liquor = pd.read_csv('Iowa_Liquor_Sales_Sample_10pct.csv')

In [2]:
liquor
liquor.isnull().sum()

Date                        0
Store Number                0
City                        0
Zip Code                    0
County Number            1077
County                   1077
Category                   68
Category Name             632
Vendor Number               0
Item Number                 0
Item Description            0
Bottle Volume (ml)          0
State Bottle Cost           0
State Bottle Retail         0
Bottles Sold                0
Sale (Dollars)              0
Volume Sold (Liters)        0
Volume Sold (Gallons)       0
dtype: int64

In [ ]:
# so about 1100 missing county names/numbers, and 600 missing category names. They're mildly annoying,
# but don't impact the statistical value of revenue, really.

In [3]:
# Those column names are ugly and difficult. Let's make them more manageable.

names = [u'Date', u'Store_Number', u'City', u'Zip_Code', u'County_Number',
       u'County', u'Category', u'Category_Name', u'Vendor_Number',
       u'Item_No', u'Item_Desc', u'Volume_mL',
       u'State_Cost', u'State_Retail', u'Bottles_Sold',
       u'Sale_val', u'Volume_Sold_L', u'Volume_Sold_G']
liquor.columns = names

In [4]:
# Make sure that the above did what it was supposed to do
liquor.head()

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Vendor_Number,Item_No,Item_Desc,Volume_mL,State_Cost,State_Retail,Bottles_Sold,Sale_val,Volume_Sold_L,Volume_Sold_G
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0,5.55


In [5]:
# slice off the $
liquor['State_Cost'] = liquor.State_Cost.str.replace('$','')
liquor['State_Retail'] = liquor.State_Retail.str.replace('$','')
liquor['Sale_val'] = liquor['Sale_val'].str.replace('$','')

In [6]:
# convert strings to floats for aggregation
liquor['State_Cost'] = liquor.State_Cost.astype(float)
liquor['State_Retail'] = liquor.State_Retail.astype(float)
liquor['Sale_val'] = liquor['Sale_val'].astype(float)

In [7]:
# convert date to date/time
liquor['Date'] = pd.to_datetime(liquor.Date)

In [8]:
# Split out the year, month to get a bit more granularity
liquor['Month'] = liquor['Date'].dt.month
liquor['Year'] = liquor['Date'].dt.year

In [ ]:
# # convert unnecessary floats to strings; this made sense at the time. I was trying to do
# something specific here, but only upsets Jupyter afterwarads

# liquor['Store_Number'] = liquor.Store_Number.astype(str)
# liquor['County_Number'] = liquor.County_Number.astype(str)
# liquor['Vendor_Number'] = liquor.Vendor_Number.astype(str)
# liquor['Item_No'] = liquor.Item_No.astype(str)
# liquor.dtypes

In [ ]:
# There seem to be a lot of unnecessary columns here. Although it's difficult to say what is predictive,
# a few stand out as clearly not. E.g., the volume of a given bottle, the vendor number, the item number
# We're interested in total sales, by store, and there's not a lot here that is likely to be predictive

In [9]:
liquor.drop(['Vendor_Number', 'Item_No', 'Volume_mL'], axis=1, inplace=True)

In [12]:
# Just out of curiosity, what's the state's markup on the hooch?
liquor['State_Margin'] = ((liquor.State_Retail - liquor.State_Cost) / liquor.State_Retail)

In [14]:
liquor['State_Margin'].describe()

count    270955.000000
mean          0.334214
std           0.004976
min           0.000000
25%           0.333333
50%           0.333333
75%           0.333603
max           0.714286
Name: State_Margin, dtype: float64

In [16]:
profit = liquor['State_Margin'].median()
profit

0.33333333333333337

In [ ]:
# Mean, median state margin is 33%, with almost no standard deviation

In [17]:
#Rounding because we are dealing with currency; it makes sense to 
liquor.round(2)

,Date,Store_Number,City,Zip_Code,County_Number,County,Category,Category_Name,Item_Desc,State_Cost,State_Retail,Bottles_Sold,Sale_val,Volume_Sold_L,Volume_Sold_G,Month,Year,State_Margin
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,Mr. Boston Apricot Brandy,4.50,6.75,12,81.00,9.00,2.38,11,2015,0.33
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,Tin Cup,13.75,20.63,2,41.26,1.50,0.40,3,2016,0.33
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,Jim Beam,12.59,18.89,24,453.36,24.00,6.34,2,2016,0.33
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,1800 Ultimate Margarita,9.50,14.25,6,85.50,10.50,2.77,2,2016,0.33
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,Five O'clock Vodka,7.20,10.80,12,129.60,21.00,5.55,8,2015,0.33
5,2015-04-20,2569,CEDAR RAPIDS,52402,57.0,Linn,1041100.0,AMERICAN DRY GINS,New Amsterdam Gin,13.32,19.98,6,119.88,10.50,2.77,4,2015,0.33
6,2015-08-05,2596,OTTUMWA,52501,90.0,Wapello,1051010.0,AMERICAN GRAPE BRANDIES,Korbel Brandy,6.66,9.99,3,29.97,2.25,0.59,8,2015,0.33
7,2015-06-25,3456,CLEAR LAKE,50428,17.0,Cerro Gordo,1012100.0,CANADIAN WHISKIES,Canadian Club Whisky,15.75,23.63,2,47.26,3.50,0.92,6,2015,0.33
8,2016-01-04,4757,BONDURANT,50035,77.0,Polk,1032080.0,IMPORTED VODKA,Absolut Swedish Vodka 80 Prf,11.49,17.24,4,68.96,3.00,0.79,1,2016,0.33
9,2015-11-10,4346,SHELLSBURG,52332,6.0,Benton,1081315.0,CINNAMON SCHNAPPS,Dekuyper Hot Damn!,7.62,11.43,2,22.86,2.00,0.53,11,2015,0.33


In [18]:
# Records for 2016
liquor_16 = liquor[(liquor.Year == 2016)]

In [49]:
liquor_16.Date.value_counts()

2016-01-04    1311
2016-01-25    1296
2016-03-14    1219
2016-01-05    1208
2016-03-09    1169
2016-03-22    1168
2016-03-15    1164
2016-02-02    1161
2016-03-16    1150
2016-03-28    1138
2016-01-06    1126
2016-02-10    1121
2016-03-07    1120
2016-03-21    1115
2016-02-24    1112
2016-01-27    1110
2016-02-01    1106
2016-03-30    1103
2016-03-02    1102
2016-02-23    1100
2016-02-03    1100
2016-02-17    1099
2016-02-16    1097
2016-02-08    1094
2016-01-20    1094
2016-01-26    1076
2016-01-19    1070
2016-03-08    1054
2016-02-22    1054
2016-02-15    1046
2016-01-11    1045
2016-03-01    1041
2016-02-09    1039
2016-02-29    1037
2016-03-23    1031
2016-01-13    1019
2016-01-12    1012
2016-03-29     984
2016-01-07     842
2016-03-10     836
2016-03-03     808
2016-02-25     783
2016-02-11     769
2016-03-24     746
2016-02-04     742
2016-02-18     730
2016-01-21     713
2016-03-17     711
2016-01-14     703
2016-03-31     670
2016-01-28     665
2016-01-15     551
2016-03-04  

In [19]:
# Records for 2015
liquor_15 = liquor[(liquor.Year == 2015)]

In [20]:
# Monthly total revenue for 2015
liquor_15_monthly = liquor_15.groupby('Month').Sale_val.agg(['sum'])
liquor_15_monthly

,sum
Month,
1,1858000.63
2,2037903.48
3,2257891.78
4,2302566.62
5,2259716.10
6,2754960.95
7,2175922.14
8,2164753.72
9,2387918.92


In [21]:
#Split 2015 into Quarters
q1_15 = liquor_15[(liquor_15.Month <= 3)]
q2_15 = liquor_15[((liquor_15.Month > 3) & (liquor_15.Month <= 6))]
q3_15 = liquor_15[((liquor_15.Month < 6) & (liquor_15.Month <=9))]
q4_15 = liquor_15[(liquor_15.Month > 9)]

In [ ]:
# The next two cells are playing around with transpose and stack/unstack. I was planning to check revenue by store 
# month by month before I realized that the aggregated figures would just work better. I did, ultimately, get each 
# store on a single line, with each month's revenue broken out. Rather pleased with that.

In [ ]:
# liquor_15_monthly.transpose()

In [ ]:
# mo_sale_15 = liquor_15.groupby(['Store_Number', 'Month']).Sale_val.agg(['sum']).transpose()
# s = mo_sale_15.stack().transpose()

In [22]:
# Aggregating mean volume sold per store, annually
mean_vol_15 = q1_15.groupby(['Store_Number']).Volume_Sold_L.agg(['mean'])

In [23]:
# Total for liquor sales in 2015 across all stores
sales_15 = round(sum(liquor_15.Sale_val), 2)
sales_15

28527245.39

In [24]:
# Estimated State profit for liquor sales in 2015
profit_15 = sales_15 * profit
round(profit_15,2)

9509081.8

In [26]:
# Quarterly Sales by store, q1 15
q1_15_stores = q1_15.groupby('Store_Number').Sale_val.agg(['sum'])

In [ ]:
# Mean quarterly liquor sales, by store
q1_15.groupby('Store_Number').Volume_Sold_L.agg(['mean'])

In [28]:
# Summing store sales in 15
store_sales_15 = liquor_15.groupby('Store_Number').Sale_val.agg(['sum'])

In [29]:
# Begin setting up a series of dataframes from the GroupBy objects in order to build the data frame that I required
# Considering how repetitive it is, there is almost certainly a way to write a function that does this. Something
# to think about later.
store_15 = ['Annual_Sales_15']

store_sales_15.columns = store_15
q1_15 = ['Q1_15_sales']
q1_15_stores.columns = q1_15
df1 = pd.merge(store_sales_15, q1_15_stores, left_index=True, right_index=True, how='outer')

In [ ]:
# confirm that the first constructed dataframe is what I need it to be
df1.dtypes

In [30]:
# Second dataframe for merging
annl_vol_15 = liquor_15.groupby('Store_Number').Volume_Sold_L.agg(['sum'])
df2 = pd.merge(df1, annl_vol_15, left_index=True, right_index=True, how='outer')
df2.rename(columns = {'sum':'Annual_Volume'}, inplace=True)

In [31]:
# Third dataframe, initial modeling data
df3 = pd.merge(df2, mean_vol_15, left_index=True, right_index=True, how='outer')
df3

,Annual_Sales_15,Q1_15_sales,Annual_Volume,mean
Store_Number,,,,
2106,146326.22,39287.29,9731.85,19.582171
2113,9310.22,2833.25,659.85,4.216905
2130,111871.43,24272.57,6891.37,16.635057
2152,7721.08,2003.46,633.37,4.741875
2178,24324.18,5856.41,1917.12,8.537708
2190,121689.06,29452.92,6322.17,4.802824
2191,125093.49,29085.57,8053.32,12.962119
2200,22811.55,4900.43,1817.24,4.377619
2205,24681.39,6407.74,1556.91,5.362571


In [32]:
# Since our test set is only q1 See how things correlate here
df3.corr()

,Annual_Sales_15,Q1_15_sales,Annual_Volume,mean
Annual_Sales_15,1.000000,0.981456,0.992339,0.356906
Q1_15_sales,0.981456,1.000000,0.972804,0.385705
Annual_Volume,0.992339,0.972804,1.000000,0.377593
mean,0.356906,0.385705,0.377593,1.000000


In [33]:
df3.fillna(value=0, inplace=True)

In [34]:
names3 = [u'Annual_Sales_15', u'Q1_15_sales', u'Annual_Volume', u'Q1_Mean_LVol_Mo']
df3.columns = names3

In [35]:
df3.shape

(1372, 4)

# Modeling Modeling now we go..

In [36]:
linr = LinearRegression()

In [44]:
# Best Fit Model for 2015, based on the variables I have selected
features = ['Q1_15_sales','Q1_Mean_LVol_Mo']
target = ['Annual_Sales_15']
X = df3[features]
y = df3[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=76)
model = linr.fit(X_train,y_train)
y_predicts = model.predict(X_test)

print np.sqrt(mean_squared_error(y_test, y_predicts))


8376.11406773


In [ ]:
# And, since the object is to predict the annual sales for 2016, we take our lovely little model and apply it to 
# q1 sales for 2016

In [55]:
store_sales_16 = liquor_16.groupby('Store_Number').Sale_val.agg(['sum'])
store_sales_16.head(4)

,sum
Store_Number,
2106,30523.75
2113,2065.90
2130,27856.11
2152,1376.43


In [56]:
vol_16 = liquor_16.groupby('Store_Number').Volume_Sold_L.agg(['mean'])
vol_16.head(3)

,mean
Store_Number,
2106,16.675197
2113,4.783784
2130,13.306838


In [59]:
df_mod = pd.merge(store_sales_16, vol_16, left_index=True, right_index=True, how='outer')

df_mod.columns

Index([u'sum', u'mean'], dtype='object')

In [62]:
df_mod['16_annl'] = store_sales_16 * 4

In [63]:
# Train the model on the Q1 16 data and get some predictions
X_ind = df_mod[['sum', 'mean']]
y_dep = df_mod['16_annl']
linr.fit(X_ind, y_dep)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [65]:
Annl_Pred = linr.predict(X_ind)

In [67]:
Annl_Pred = sum(Annl_Pred) * 4

In [68]:
Annl_Pred

102391960.95999965

# Just for fun and games: Most popular category by year

In [ ]:
liquor.groupby(['Year','Category_Name']).Volume_Sold_L.agg(['sum']).sort_values(by='sum', ascending=False).head(10)